# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [30]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [31]:
# temp code
# import os

# def count_files_and_folders(directory):
#     num_files = 0
#     num_folders = 0

#     for root, dirs, files in os.walk(directory):
#         num_folders += len(dirs)
#         num_files += len(files)

#     return num_files, num_folders
# # /content/drive/My Drive/AI/Upgrad/Melanoma_Assignment_CNN/CNN_assignment/Skin cancer ISIC The International Skin Imaging Collaboration/Train
# # Example usage
# directory = '/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/Project_data/train'  # Replace with your directory path
# files, folders = count_files_and_folders(directory)
# print(f"Number of files: {files}")
# print(f"Number of folders: {folders}")

# directory = '/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/Project_data/val'  # Replace with your directory path
# files, folders = count_files_and_folders(directory)
# print(f"Number of files: {files}")
# print(f"Number of folders: {folders}")

In [32]:
import numpy as np
import os
import matplotlib.pyplot as plt

# from scipy.misc import imread, imresize
from imageio import imread
from PIL import Image
# Resize using PIL.Image.resize
# image = Image.fromarray(image)  # Convert to PIL Image
# image = image.resize((100, 100))  # Resize to (100, 100)
# image = np.array(image)  # Convert back to NumPy array

import datetime
import os

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv3D, MaxPooling3D , Flatten, Dropout, GlobalAveragePooling3D

In [33]:
#!pip install --upgrade tensorflow

We set the random seed so that the results don't vary drastically.

In [34]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf

#tf.set_random_seed(30)
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [35]:
train_doc = np.random.permutation(open('/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/Project_data/val.csv').readlines())

#experiment with the batch size
batch_size = 100


In [36]:
type(train_doc)

numpy.ndarray

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [37]:
#source_path = '/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/Project_data'
#folder_list = train_doc,val_doc above
#batch_size = batch_size above
def generator(source_path, folder_list, batch_size):
    # print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [i for i in range(0,30,2)]
    # img_idx = [i for i in range(0,30,1)]
    while True:
        # print('Here 1')
        # print(folder_list)

        t = np.random.permutation(folder_list) #temp comment

        # calculate the number of batches
        num_batches = int(len(folder_list)/batch_size)
        # print('num_batches',num_batches)

        for batch in range(num_batches): # we iterate over the number of batches
            # print('111')
            batch_data = np.zeros((batch_size,len(img_idx),120,160,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                # print('222')
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                # print(t[folder])
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    # print('333')
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #crop the images and resize them. Note that the images are of 2 different shape
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = Image.fromarray(image.astype(np.uint8))  # Convert to PIL Image
                    image = image.resize((160, 120))  # Resize to (100, 100)
                    image = np.array(image)  # Convert back to NumPy array

                    # print(image[:,:,0],image[:,:,0],image[:,:,2])
                    batch_data[folder,idx,:,:,0] = image[:,:,0]/255.0
                    batch_data[folder,idx,:,:,1] = image[:,:,1]/255.0
                    batch_data[folder,idx,:,:,2] = image[:,:,2]/255.0

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


        # write the code for the remaining data points which are left after full batches
        remaining_data_points = len(folder_list) - batch_size*num_batches
        batch_data = np.zeros((batch_size,len(img_idx),120,160,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
        for folder in range(len(folder_list)-remaining_data_points,len(folder_list)): # iterate over the batch_size
            counter = 0
            # print('remaining: ',t[folder])
            imgs = os.listdir(source_path+'/'+ t[folder].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = imread(source_path+'/'+ t[folder].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                #crop the images and resize them. Note that the images are of 2 different shape
                #and the conv3D will throw error if the inputs in a batch have different shapes
                image = Image.fromarray(image.astype(np.uint8))  # Convert to PIL Image
                image = image.resize((160,120))  # Resize to (100, 100)
                image = np.array(image)  # Convert back to NumPy array

                # print(image[:,:,0]/255.0,image[:,:,0]/255.0,image[:,:,2]/255.0)
                batch_data[counter,idx,:,:,0] = image[:,:,0]/255.0
                batch_data[counter,idx,:,:,1] = image[:,:,1]/255.0
                batch_data[counter,idx,:,:,2] = image[:,:,2]/255.0

            batch_labels[counter, int(t[counter].strip().split(';')[2])] = 1
            counter += 1
        yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do





In [38]:
# # temp code
# train_doc2 = open('/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/temp/train.csv').readlines()
# val_doc2 = open('/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/temp/val.csv').readlines()

In [39]:
# # temp code
# source_path = '/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/temp/train'
# # source_path = '/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/Project_data/train'
# folder_list = train_doc2
# batch_size = 4
# generator(source_path, folder_list, batch_size)

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [40]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/Project_data/train'
val_path = '/content/drive/My Drive/AI/Upgrad/Gesture_Recognition_CaseStudy/Project_data/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [41]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Conv3D, MaxPooling3D, GlobalAveragePooling3D, Dropout
# from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here


In [42]:
img_idx = [i for i in range(0,30,2)]
# img_idx = [i for i in range(0,30,1)]
len(img_idx)

15

In [43]:
model = Sequential()

model.add(Conv3D(32, (3,3,3), padding='same', activation='relu', input_shape=(len(img_idx),120, 160, 3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(128, (3,3,3), padding='same', activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

# model.add(Conv3D(256, (3,3,3), padding='same', activation='relu'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
model.add(GlobalAveragePooling3D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [44]:
optimiser = 'adam'
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_6 (Conv3D)                    │ (None, 15, 120, 160, 32)    │           2,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_6 (MaxPooling3D)       │ (None, 7, 60, 80, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 7, 60, 80, 32)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_7 (Conv3D)                    │ (None, 7, 60, 80, 64)       │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_7 (MaxPooling3D)       │ (None, 3, 30, 40, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 3, 30, 40, 64)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_8 (Conv3D)                    │ (None, 3, 30, 40, 128)      │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_8 (MaxPooling3D)       │ (None, 1, 15, 20, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 1, 15, 20, 128)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling3d_2           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling3D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 313,605 (1.20 MB)

 Trainable params: 313,605 (1.20 MB)

 Non-trainable params: 0 (0.00 B)

None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [45]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [46]:
batch_size

100

In [47]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

if not os.path.exists(model_name):
    os.mkdir(model_name)

filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.keras'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='auto', min_lr=0.00001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [48]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [49]:
num_train_sequences

663

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [50]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=0,
                    callbacks=callbacks_list, validation_data=val_generator,
                    validation_steps=validation_steps, class_weight=None,  initial_epoch=0)

<ipython-input-37-6b856358b419>:29: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
<ipython-input-37-6b856358b419>:54: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(source_path+'/'+ t[folder].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)



Epoch 1: saving model to model_init_2025-01-0213_19_18.917483/model-00001-1.38849-0.16857-1.60602-0.23000.keras

Epoch 2: saving model to model_init_2025-01-0213_19_18.917483/model-00002-1.38106-0.17571-0.00000-0.00000.keras

Epoch 3: saving model to model_init_2025-01-0213_19_18.917483/model-00003-1.37469-0.16857-1.59189-0.23000.keras

Epoch 4: saving model to model_init_2025-01-0213_19_18.917483/model-00004-1.35366-0.19571-0.00000-0.00000.keras

Epoch 5: saving model to model_init_2025-01-0213_19_18.917483/model-00005-1.34206-0.19000-1.50528-0.24000.keras

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 6: saving model to model_init_2025-01-0213_19_18.917483/model-00006-1.30741-0.20143-0.00000-0.00000.keras

Epoch 7: saving model to model_init_2025-01-0213_19_18.917483/model-00007-1.26873-0.25143-1.39311-0.30000.keras

Epoch 8: saving model to model_init_2025-01-0213_19_18.917483/model-00008-1.23257-0.29143-0.00000-0.00000.keras

Epoch 8: ReduceLRO